# DHQ Author Scrape:
- Iterate through each volume/article of DHQ
- Write year, vol., num., title and authors to .csv file

In [ ]:
from bs4 import BeautifulSoup
import re
import time
import requests

r  = requests.get("http://www.digitalhumanities.org/dhq/")
soup = BeautifulSoup(r.content, "lxml")
#print(soup.prettify())

nav = soup.find("div", {"id": "leftsidenav"})
index = nav.find_all('a')

#Extract URL of each Issue
for issues in index:
    issue = issues.get('href')
    r2  = requests.get("http://www.digitalhumanities.org" + issue)
    soup = BeautifulSoup(r2.content, "lxml")
    
    #Extract URL of each Article
    for article in soup.find_all("div", {"class": "articleInfo"}):
        link = article.find('a')
        article2 = link.get('href')
        #print(article2)

        #Get Article
        r2  = requests.get("http://www.digitalhumanities.org" + article2)
        soup = BeautifulSoup(r2.content, "lxml")

        #Get Title
        title = soup.title.get_text()
        newtitle = title.strip()
        replaced = re.sub('\s{2,}', ' ', newtitle)
        t = re.search('y:\s(.*)', replaced)
        title2 = t.group(1)
        #print(title2)

        #Get Year, Volume, Number
        v = re.search('vol\/(\d+)', article2)
        vol = v.group(1)
        #print(vol)
        n = re.search('vol\/\d+\/(\d+)', article2)
        num = n.group(1)
        #print(num)
        year = int(vol) + 2006
        #print(year)

        #Get Author Names
        a_list = []
        for names in soup.find_all("div", {"class": "author"}):
            name = names.find('a').get_text()
            name = name.strip()
            name = re.sub('\s{2,}', ' ', name)
            a_list.append(name)
        a_list = ','.join(a_list)
        #print(a_list)
        #len(a_list)

        final = {
            "title" : title2,
            "year" : year,
            "volume" : vol,
            "number" : num,
            "authors" : a_list,
            }
        print(final)
        
        #Write to .csv file
        import csv
        with open('DHQ.csv', 'a') as csvfile:
            fieldnames = ['year', 'volume', 'number', 'title', 'authors']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            #writer.writeheader()
            writer.writerow({
            "title" : title2,
            "year" : year,
            "volume" : vol,
            "number" : num,
            "authors" : a_list,})
            time.sleep(1)